# Case de Processo Seletivo: IBOPE Inteligência

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [0]:
xls = 'Case - Data Science IBOPE.XLS'

In [495]:
dados = pd.read_excel(xls, sheet_name='Informações_Municipais', header=3)
dados = dados.rename(columns={'Taxa bruta de freqüência à escola, 2000':'Taxa bruta de frequência à escola, 2000'})
dados = dados.rename(columns={'Média de anos de estudo das pessoas de 25 anos ou mais de idade, 2000':'Média de anos de estudo das pessoas de 25 anos ou mais, 2000'})
dados.sample(3)

,Código,Município,Área (km²),"Densidade demográfica, 2000",Distância à capital (km),"Esperança de vida ao nascer, 2000","Mortalidade até um ano de idade, 2000","Taxa de fecundidade total, 2000","Percentual de pessoas de 25 anos ou mais analfabetas, 2000","Renda per Capita, 2000","Índice de Gini, 2000","Intensidade da indigência, 2000","Intensidade da pobreza, 2000","Índice de Desenvolvimento Humano Municipal, 2000","Taxa bruta de frequência à escola, 2000","Taxa de alfabetização, 2000","Média de anos de estudo das pessoas de 25 anos ou mais, 2000","População de 25 anos ou mais de idade, 1991","População de 25 anos ou mais de idade, 2000","População de 65 anos ou mais de idade, 1991","População de 65 anos ou mais de idade, 2000","População total, 1991","População total, 2000","População urbana, 2000","População rural, 2000"
1045,420390,Capinzal (SC),224.8,88.8,303.282108,73.73,16.34,2.20,9.34,263.41,0.50,62.12,42.61,0.813,91.60,93.00,5.16,7293,10290,634,921,15562,19955,15460,4495
4572,354890,São Carlos (SP),1143.9,168.7,213.153830,73.08,12.57,2.42,7.33,456.25,0.52,67.91,38.23,0.841,89.61,94.36,7.40,84178,109677,10758,14279,158221,192998,183433,9565
2067,291200,Ibiassucê (BA),383.5,33.4,429.115723,68.08,32.39,2.31,38.39,104.32,0.54,56.88,53.44,0.659,71.89,70.66,2.76,5104,6492,751,1010,11898,12828,4037,8791


### Adicionando novos dados ao Dataset

In [496]:
dados2 = dados.copy()
dados2['TAXA DE CRESCIMENTO: MAIS DE 25 ANOS'] = dados['População de 25 anos ou mais de idade, 2000'] - dados['População de 25 anos ou mais de idade, 1991']
dados2['TAXA DE CRESCIMENTO: MAIS DE 65 ANOS'] = dados['População de 65 anos ou mais de idade, 2000'] - dados['População de 65 anos ou mais de idade, 1991']
dados2['TAXA DE CRESCIMENTO POPULACIONAL EM 10 ANOS'] = dados['População total, 2000'] - dados['População total, 1991']
dados2['PROPORÇÃO URBANA DA POPULAÇÃO'] = dados['População urbana, 2000'] / dados['População total, 2000']
dados2['PROPORÇÃO RURAL DA POPULAÇÃO'] = dados['População rural, 2000'] / dados['População total, 2000']
dados = dados2

dados.shape # Tamanho do dataset

(5507, 30)

### Verificação de dados nulos no Dataset

In [497]:
# Tratar os dados/fazer as taxas
dados.info() # todos os dados estão preenchidos.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5507 entries, 0 to 5506
Data columns (total 30 columns):
Código                                                          5507 non-null int64
Município                                                       5507 non-null object
Área (km²)                                                      5507 non-null float64
Densidade demográfica, 2000                                     5507 non-null float64
Distância à capital (km)                                        5507 non-null float64
Esperança de vida ao nascer, 2000                               5507 non-null float64
Mortalidade até um ano de idade, 2000                           5507 non-null float64
Taxa de fecundidade total, 2000                                 5507 non-null float64
Percentual de pessoas de 25 anos ou mais analfabetas, 2000      5507 non-null float64
Renda per Capita, 2000                                          5507 non-null float64
Índice de Gini, 2000                

In [0]:
# Os dados estão 100%

### Criação de dicionários

Estes dicionários tem como objetivo facilitar a visualização dos dados, transformando as diversas colunas em valores numéricos

In [0]:
dicionario, dicionario_inverso = {}, {}
for item in dados.columns: 
  dicionario[item] = list(dados.columns).index(item)
  dicionario_inverso[list(dados.columns).index(item)] = item
#dicionario_inverso

In [0]:
def func_prepara_para_padronizar(dados): #organiza os dados para clusterização
  return dados.drop(["Código", "Município"], axis = 1).rename(columns=dicionario)

In [501]:
dados_numericos = func_prepara_para_padronizar(dados); dados_numericos.head()

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,136.9,36.3,21.414308,67.58,27.70,2.85,15.64,205.27,0.53,36.89,34.40,0.742,80.17,88.03,4.87,1872,2433,121,227,4227,4971,3096,1875,561,106,744,0.622812,0.377188
1,897.4,7.2,390.876513,72.92,20.51,2.54,14.48,196.36,0.52,51.90,35.22,0.760,74.57,86.58,4.36,3308,3781,354,459,6492,6446,3927,2519,473,105,-46,0.609215,0.390785
2,1047.7,10.9,78.641435,69.59,22.36,2.77,21.81,168.31,0.56,47.06,40.32,0.723,74.18,82.45,4.24,4241,5918,446,686,9402,11452,7206,4246,1677,240,2050,0.629235,0.370765
3,1613.9,73.8,51.184552,70.77,25.61,3.15,24.85,102.14,0.59,44.61,51.04,0.706,81.76,80.74,4.28,33554,44897,3792,4688,99989,119152,70843,48309,11343,896,19163,0.594560,0.405440
4,1822.4,12.3,172.311549,72.50,21.59,2.41,15.56,253.68,0.58,41.53,31.68,0.778,78.39,87.55,4.78,10163,12406,1084,1595,20689,22360,19022,3338,2243,511,1671,0.850716,0.149284


###Padronização dos dados

In [0]:
from sklearn.preprocessing  import StandardScaler

scaler = StandardScaler()
dados_padronizados = scaler.fit_transform(dados_numericos)
# dados_padronizados

### K-Means: Busca pelo melhor número de clusters (Método do cotovelo)

O método Elbow se trata de uma técnica interessante para encontrar o valor ideal do parâmetro k.
Basicamente o que o método faz é testar a variância dos dados em relação ao número de clusters.
É considerado um valor ideal de k quando o aumento no número de clusters não representa um valor significativo de ganho.

In [0]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [0]:
X = np.array(dados_padronizados)#; X

In [0]:
K = range(1,15)

distorcoes = []

for k in K:
  kmeans_modelo = KMeans(n_clusters = k).fit(X)
  kmeans_modelo.fit(X)
  distorcoes.append(sum(np.min(cdist(X, kmeans_modelo.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

plt.plot(K, distorcoes, 'bx-')
plt.xlabel('k')
plt.ylabel('Distorção')
plt.show()

### Dendrograma

A decisão do número de clusters que melhor representem diferentes grupos podem ser escolhidos observando-se o dendrograma. A melhor escolha do número de clusters é o número de linhas verticais no dendrograma cortadas por uma linha horizontal que pode atravessar verticalmente a distância máxima sem intersectar um cluster.

In [0]:
from scipy.cluster.hierarchy import dendrogram, linkage # dendrograma

modelo = KMeans(n_clusters=15)
modelo.fit(X)

grupos = pd.DataFrame(modelo.cluster_centers_,
             columns=dados_numericos.columns)

matriz_de_distancia = linkage(grupos)

dendrograma = dendrogram(matriz_de_distancia)
dendrograma;

### Criação do modelo K-Means

A ideia aqui é fazer uma segunda versão abrangindo mais de 3 categorias, para deixar maior a variação e percepção dos dados, diferente de uma tentativa prévia, e visualizar de uma forma um pouco mais exponêncial um potencial crescimento.

In [0]:
n = 7 # número de clusters
#n=3

modelo = KMeans(n_clusters=n, random_state=0)
modelo.fit(X)

In [0]:
# quantidade em cada grupo
pd.Series(modelo.labels_).value_counts()

### Visualização dos grupos

In [0]:
grupos = pd.DataFrame(modelo.cluster_centers_, columns=dados_numericos.columns)
grupos.transpose().plot.bar(
    subplots = True, figsize=(20, 15), sharex = False, rot=0);

In [0]:
#grupos

### Adição do grupo a tabela

In [0]:
dados_agrupados = dados.copy()
dados_agrupados['GRUPOS'] = modelo.labels_#; dados_agrupados.sample(3)

### Análise por grupo: média

In [0]:
número_de_municipios = pd.Series(modelo.labels_).value_counts().sort_index()

In [0]:
medias_grupos = dados_agrupados.drop("Código", axis=1).set_index("GRUPOS").groupby("GRUPOS").aggregate(np.mean).round(2)
medias_grupos["Quantidade de cidades no grupo"] = número_de_municipios; medias_grupos

In [0]:
#medias_grupos.sort_values("População total, 2000", ascending=False)

Nesta análise é possivel notar uma relação bem clara entre os 4 primeiros grupos, onde há uma relação direta com o crescimento de renda, população e desenvolvimento humano no geral. Há uma relação direta entre os melhores lugares a ser investidos: onde há mais clientes e onde a renda é superior. Vou utilizar um dicionário para guardar esta sequência.

In [0]:
########## verificar
dicionario_nivel_cidade = {3:'A', 2:'B', 5:'C', 0:'D', 1:'E', 6:'F', 4:'G'}
###########

In [0]:
medias_grupos.sort_values("População total, 2000", ascending=False).rename(index=dicionario_nivel_cidade)

### Principais parâmetros por grupos:

In [0]:
grupos_renomeados = grupos.copy()
grupos_renomeados = grupos_renomeados.rename(columns=dicionario_inverso)#; grupos_renomeados

In [0]:
# Verifica características relevantes de um determinado grupo
grupo_para_verificar = 2
grupos_renomeados.loc[grupo_para_verificar].sort_values(ascending = False)

In [0]:
dados_agrupados[dados_agrupados['GRUPOS'] == grupo_para_verificar].sample(5) # verifica a tabela por grupo

### Modelo Preditivo

In [0]:
# Previsão pelo modelo
amostra = dados.sample()
amostra_drop = func_prepara_para_padronizar(amostra)
amostra_padronizada = scaler.transform(amostra_drop)#; amostra_padronizada

In [0]:
#print(modelo.predict(amostra_padronizada))
#print(dados_agrupados['GRUPOS'].iloc[amostra_drop.index].values)

In [0]:
def probabilidade_de_proximidade(amostra_padronizada):
  distancias = modelo.transform(amostra_padronizada)
  distancias2 = 1/distancias
  soma = distancias2.sum()
  proporcoes = distancias2 / soma

  grupo = 0
  for item in proporcoes[0]:
    print(f'grupo {dicionario_nivel_cidade[grupo]} ou {grupo}: {round(item*100,2)}%')
    grupo += 1

In [0]:
x = probabilidade_de_proximidade(amostra_padronizada); x